# [2021 W40 | Tableau: Can You Make Stacked Bar Charts Easier to Compare?](http://www.workout-wednesday.com/tab2021w40/)

>**Table of contents:**
>
>&ensp;&ensp;[Introduction](#Introduction) <br>
>&ensp;&ensp;[Instructions](#Instructions) <br>
>&ensp;&ensp;[Workings](#Workings) <br>
>&ensp;&ensp;[Results](#Results)


### Introduction
As the [author](http://www.workout-wednesday.com/author/candra/) mentions in the [challenge's page](http://www.workout-wednesday.com/tab2021w40/), it is hard to compare between the components in a stacked bar chart, except for the bottom component. 

The task is to create a vertical stacked bar chart (sales by category) that allows the user to select and switch the category to the bottom.
    
The [dataset](https://data.world/cmack624/superstore-v20212) used in the challenge is the superstore dataset for Tableau 2021.2. <br>
However, I found the `State` column is missing in the dataset provided and used the [superstore dataset](https://data.world/cmack624/superstore-20204) for Tableau 2021.4 instead.

The solution provided in the challenge's page is shown below.

![solution](https://drive.google.com/uc?export=view&id=13csiFibWQ_tY9oi2jQnFTWdwNVijSiv1)

In the solution provided, the user can click the category on the chart and it will be moved to the bottom.

### Instructions

- Separate the states based on their beginning letter, namely A-M, N-Z.

- Create a vertical stacked bar chart (sales by category) that switches the selected category to the bottom.

- Non-selected bar components should not be greyed out if something else is highlighted.

- No drop down menus.

- Match the Formatting.

### Workings

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
csv_path = 'https://raw.githubusercontent.com/ywjet/Data-Visualization/main/Data/2021%20Week%2014_Tableau_Can%20You%20Recommend%20Profitable%20Return%20Customer%20Bundles.csv'

# reading data
df = pd.read_csv(csv_path, usecols=['Category', 'State', 'Sales'])
df.head()

,State,Category,Sales
0,Kentucky,Furniture,261.9600
1,Kentucky,Furniture,731.9400
2,California,Office Supplies,14.6200
3,Florida,Furniture,957.5775
4,Florida,Office Supplies,22.3680


In [3]:
# retain only the first letter of the state
df['State'] = [state[0] for state in df['State']]
df.head()

,State,Category,Sales
0,K,Furniture,261.9600
1,K,Furniture,731.9400
2,C,Office Supplies,14.6200
3,F,Furniture,957.5775
4,F,Office Supplies,22.3680


In [4]:
# sort State & reindex
df = df.sort_values('State', ignore_index=True)
# get the index of the first N occurs
first_N = df[df['State'] == 'N'].index[0]

# separate the states based on their beginning letter
df.loc[:first_N-1]['State'] = 'A-M'
df.loc[first_N:]['State'] = 'N-Z'

In [5]:
# compute the total sales based on State & Category
df2 = df.groupby(['State','Category']).sum()
df2.reset_index(inplace=True)

# custom text to be shown on the bars
df2['Text'] = ['$'+str(round(sales, -3))[:-5]+'K' for sales in df2['Sales']]
df2

,State,Category,Sales,Text
0,A-M,Furniture,352580.9045,$353K
1,A-M,Office Supplies,381184.1580,$381K
2,A-M,Technology,394487.3420,$394K
3,N-Z,Furniture,389418.8908,$389K
4,N-Z,Office Supplies,337862.8740,$338K
5,N-Z,Technology,441666.6910,$442K


In [6]:
# total sales (will be used as y-coordinate later)
y0 = df2[df2['State'] == 'A-M']['Sales'].sum()
y1 = df2[df2['State'] == 'N-Z']['Sales'].sum()

# format total sales as str, e.g. $1,234K
total_sales_A_M = round(y0, -3)
total_sales_A_M = str(f'{total_sales_A_M:,}')
total_sales_A_M = '$' + total_sales_A_M[:-6] + 'K'

total_sales_N_Z = round(y1, -3)
total_sales_N_Z = str(f'{total_sales_N_Z:,}')
total_sales_N_Z = '$' + total_sales_N_Z[:-6] + 'K'

In [7]:
import plotly
import plotly.express as px

In [8]:
def bar_chart(df):
    fig = px.bar(df, x='State', y='Sales', color='Category', text='Text',
                 custom_data=['Category'],
                 color_discrete_map={
                     'Furniture': 'steelblue',
                     'Office Supplies': 'darkorange',
                     'Technology': 'darkgrey'
                 },
                 height=500
                )

    fig.add_annotation(x=0, y=y0, 
                       text=total_sales_A_M,
                       showarrow=False,
                       yshift=10)

    fig.add_annotation(x=1, y=y1, 
                       text=total_sales_N_Z,
                       showarrow=False,
                       yshift=10)

    fig.update_traces(
        hovertemplate='Category: %{customdata[0]}<extra></extra>',
        # bar width
        width=0.6,
        
        textposition='inside',

        insidetextanchor='middle',

        textfont={'color': 'white'}
    )

    fig.update_layout(
        plot_bgcolor='white',

        yaxis={'visible':False},

        xaxis={'title':None},

        showlegend=False
    )

    return fig

In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

In [10]:
# create buttons for the user to select the category
# the selected category will be moved to the bottom
buttons = html.Div(
    [
        dbc.Button('Furniture', size='sm', style={'color':'steelblue'},
                   n_clicks=1, id='Furniture'), 
        dbc.Button('Office Supplies', size='sm', style={'color':'darkorange'},
                   n_clicks=0, id='Office Supplies'), 
        dbc.Button('Technology', size='sm', style={'color':'darkgrey'},
                   n_clicks=0, id='Technology')
    ]
)

# the web app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SIMPLEX])

app.layout = html.Div([    
    dbc.Row(
        [
            dbc.Col(
                dbc.Card(
                    [
                        dbc.CardHeader(
                            html.H4('#WOW2021 W40 | Can You Make Stacked Bar Charts Easier to Compare?'),
                            style={'color':'white',
                                   'background-color':'dimgray'}
                        ),
                        
                        dbc.CardBody(
                            [
                                html.H5(
                                    html.B('Sales by Category'),
                                    className='card-title'
                                ),
                                
                                html.H6(
                                    'Select a category to re-order the stack.', 
                                    style={'margin-bottom':'3px'}, # thin line break
                                    className='card-subtitle'
                                ),
                                
                                buttons,
                                
                                html.Br(),
                                
                                dcc.Graph(id='bar-chart')
                            ]
                        )
                    ]
                    
                ),
                
                width={"size": 4, "offset": 4})
        ],
        
        style={'margin-top':'3px'}
    )
])

@app.callback(
    Output('bar-chart', 'figure'),
    [Output('Furniture', 'n_clicks'),
     Output('Office Supplies', 'n_clicks'),
     Output('Technology', 'n_clicks')],
    [Input('Furniture', 'n_clicks'),
     Input('Office Supplies', 'n_clicks'),
     Input('Technology', 'n_clicks')]
)

def update_bar_chart(button1, button2, button3):
    dff = df2.copy()
    
    if button1:
        dff['Category'] = pd.Categorical(dff['Category'], ['Furniture', 'Technology', 'Office Supplies'])
    elif button2:
        dff['Category'] = pd.Categorical(dff['Category'], ['Office Supplies', 'Technology', 'Furniture'])
    elif button3:
        dff['Category'] = pd.Categorical(dff['Category'], ['Technology', 'Office Supplies', 'Furniture'])
    
    dff = dff.sort_values('Category')
    
    return bar_chart(dff), 0, 0, 0
    

if __name__ == '__main__':
#     app.run_server(debug=True, use_reloader=False)
    app.run_server(debug=False, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


### Results

The figure below shows the recreated solution. 

![solution](https://drive.google.com/uc?export=view&id=1hnTsu0lU-_ypuNI_WUUAyMc4c-1adLDC)

I made some buttons for the user to select the desired category. <br>
The selected category will be moved to the bottom as shown below.

![solution_gif](https://drive.google.com/uc?export=view&id=1WA5YEa67MgCTsQ9LPcIVcE92Y7hQM2se)